In [1]:
import pandas as pd

In [2]:
# UPDATE THESE TO THE PATHS TO YOUR TRAIN / VALIDATE / TEST LYRICS FILES

train_path = '/content/TRAIN lyrics data.csv'
validate_path = '/content/VALIDATE lyrics data.csv'
test_path = '/content/TEST lyrics data.csv'

In [3]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.1 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()


Saving TEST lyrics data.csv to TEST lyrics data.csv
Saving TRAIN lyrics data.csv to TRAIN lyrics data.csv
Saving VALIDATE lyrics data.csv to VALIDATE lyrics data.csv


## For Lara:

- run these models for more epochs (try 15 and see if there is improvement)
- IMPORTANT: if you see the loss on the validate set is no longer going down you can STOP training (i.e., hit the stop button on your jupyter notebook and simply run the next cell which, since load_best_model_at_end=True is specified in your TrainingArguments, will load the best checkpoint even if the error started going up!
- then test that model that was the lowest on the validate data on the test set and see if we can get lower than 0.81 that we got with 3 epochs of training


The code now is set to 15 epochs, and a batch_size of 128.

If this batch size crashes your notebook (which it did mine) then restart the notebook and try a batch size of 64.

In [5]:
# https://huggingface.co/docs/transformers/training

In [6]:
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import mean_absolute_error

In [7]:
# pip install any of the packages you don't already have, for instance...

In [8]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [9]:
pip install accelerate>=0.20.1

In [19]:
!pip install transformers

In [10]:
pip install transformers[torch]

In [11]:
pip install accelerate -U

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mae = mean_absolute_error(labels, predictions)
    return {"mae": mae}

In [13]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [14]:
# hugging face requires a custom data format:
# https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': train_path,
                                        'eval': validate_path})

# distilbert wants "text" and "labels"
# you will want to change it to whatever the model you're finetuning expects
dataset = dataset.rename_column("Lyrics", "text")
dataset = dataset.rename_column("dominance_tags", "labels")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# roberta

mae on test set after 3 train epochs = 0.8106


In [16]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/7177 [00:00<?, ? examples/s]

Map:   0%|          | 0/1538 [00:00<?, ? examples/s]

In [26]:
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'lastfm_url', 'track', 'artist', 'seeds', 'number_of_emotion_tags', 'valence_tags', 'arousal_tags', 'labels', 'mbid', 'spotify_id', 'genre', 'track edited', 'artist edited', 'text', 'HF_byline_average_sentiment', 'HF_byline_minimum_sentiment', 'HF_byline_maximum_sentiment', 'HF_byline_median_sentiment', 'HF_byline_stdv_sentiment', 'HF_byline_firstquartile_sentiment', 'HF_byline_thirdquartile_sentiment', 'HF_byline_ratio_negative', 'HF_byline_ratio_positive', 'HF_fullinput512_one_sentiment_number', 'HF_fullinput512_one_sentiment_label', 'number_lines', 'HF_ROBERTA_byline_average_sentiment', 'HF_ROBERTA_byline_minimum_sentiment', 'HF_ROBERTA_byline_maximum_sentiment', 'HF_ROBERTA_byline_median_sentiment', 'HF_ROBERTA_byline_stdv_sentiment', 'HF_ROBERTA_byline_firstquartile_sentiment', 'HF_ROBERTA_byline_thirdquartile_sentiment', 'HF_ROBERTA_byline_ratio_negative', 'HF_ROBERTA_byline_ratio_positive', 'HF_ROBERTA_fullinput

In [29]:
config = transformers.AutoConfig.from_pretrained('roberta-base')

config.num_labels = 1




In [33]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = RobertaForSequenceClassification.from_pretrained("roberta-base")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
batch_size=64
#batch_size=64 # reduce batch size due to computational restrictions, if needed

training_args = TrainingArguments(
    output_dir="lyrics_trainer_roberta",
    num_train_epochs=15,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size,   # batch size for evaluation
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    learning_rate = 1e-5,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100, ## how often to evaluate on the eval set
    save_steps=100,
    metric_for_best_model = 'eval_mae',
    load_best_model_at_end=True
  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'],
    compute_metrics=compute_metrics
)

In [ ]:
# HERE IS WHERE YOU WILL WATCH THE MAE IN THE TABLE BEING PRINTED
# if it starts going up, follow steps I write at the beginning of the notebook (under "For Lara")
trainer.train()

In [ ]:
trainer.save_model()

model.eval()

Saving model checkpoint to lyrics_trainer_roberta
Configuration saved in lyrics_trainer_roberta/config.json
Model weights saved in lyrics_trainer_roberta/pytorch_model.bin


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
predict_dataset = load_dataset('csv', data_files={'test': test_path})

predict_dataset = predict_dataset.rename_column("Lyrics without newline or punct lower", "text")
predict_dataset = predict_dataset.rename_column("dominance_tags", "labels")

tokenized_datasets = predict_dataset['test'].map(tokenize_function, batched=True)

predictions, labels, metrics = trainer.predict(tokenized_datasets, metric_key_prefix="predict")


Map: 100%|█████████████████████████| 1539/1539 [00:01<00:00, 1397.44 examples/s]
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: HF_byline_average_sentiment, number_of_emotion_tags, HF_ROBERTA_fullinput512_one_sentiment_number, text, HF_byline_maximum_sentiment, valence_tags, arousal_tags, HF_ROBERTA_byline_firstquartile_sentiment, Unnamed: 0.1, HF_byline_stdv_sentiment, mbid, track, HF_ROBERTA_byline_maximum_sentiment, artist edited, HF_byline_thirdquartile_sentiment, lastfm_url, Lyrics, number_lines, artist, HF_fullinput512_one_sentiment_number, HF_byline_ratio_negative, HF_byline_firstquartile_sentiment, HF_fullinput512_one_sentiment_label, HF_byline_ratio_positive, track edited, spotify_id, HF_ROBERTA_byline_average_sentiment, HF_ROBERTA_byline_minimum_sentiment, HF_ROBERTA_byline_stdv_sentiment, genre, HF_byline_median_sentiment, HF_ROBERTA_byline_ratio_positive, HF_ROBERTA_fullinput512_

In [ ]:
metrics

{'predict_loss': 1.186801791191101,
 'predict_mae': 0.8106750249862671,
 'predict_runtime': 289.4149,
 'predict_samples_per_second': 5.318,
 'predict_steps_per_second': 0.086}